In [19]:
!pip install noisereduce
!pip install scipy

In [7]:
# Global imports
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
from PIL import Image
import re
import noisereduce as nr

c:\Users\herna\anaconda3\envs\Tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Path to the directory containing folders with .wav files
root_path = "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"

# Define the root path for saving progress
PROGRESS_PATH = os.path.join(os.getcwd(), 'progress.txt')

In [18]:

# Function to generate spectrogram
def ggenerate_spectrogram(data, rate, save_path, resized_spectrogram_save_path):
    f, t, Sxx = signal.spectrogram(data, rate)
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(t, f, Sxx, shading='gouraud')
    plt.imshow(10 * np.log10(Sxx), aspect='auto', cmap='viridis')
    plt.axis('off')  # Turn off axis labels
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    resize_image(save_path, (792, 252), resized_spectrogram_save_path)
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='black')  # Dark background
    resize_image(save_path, (792, 252), resized_spectrogram_save_path)
    
    
# Function to generate spectrogram using scipy
def generate_spectrogram(data, rate, save_path, resized_spectrogram_save_path):
    plt.figure(figsize=(10, 4))
    f, t, Sxx = signal.spectrogram(data, fs=rate, nperseg=256, noverlap=128, window='hamming', scaling='spectrum')
    plt.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), cmap='inferno')  # Adding small value to avoid log(0)
    plt.axis('off')  # Turn off axis labels
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='black')  # Dark background
    plt.close()
    resize_image(save_path, (792, 252), resized_spectrogram_save_path)


# Function to resize image
def resize_image(image_path, target_size, save_path):
    img = Image.open(image_path)
    img_resized = img.resize(target_size, resample=Image.Resampling.NEAREST)
    img_resized.save(save_path)

In [10]:
# Function to save progress
def save_progress( wav_file_path):
    with open(PROGRESS_PATH, 'w') as file:
        file.write(wav_file_path)

# Function to load progress
def load_progress():
    progress_file = PROGRESS_PATH
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as file:
            return file.read().strip()
    else:
        return None

In [14]:
def extract_numbers_from_string(input_string):
    # Regular expression to match numbers
    pattern = r'\d+'
    # Find all numbers in the input string
    numbers = re.findall(pattern, input_string)
    return numbers

In [ ]:
# Generate preprocessed files
for folder_name in os.listdir(root_path):
  folder_path = os.path.join(root_path, folder_name)
  if os.path.isdir(folder_path):
    for wav_file_path in glob.glob(os.path.join(folder_path, '*.wav')):
      print(folder_path)
      # load data
      rate, data = wavfile.read(wav_file_path)
      # perform noise reduction
      reduced_noise = nr.reduce_noise(y=data, sr=rate)
      # Extract the audio file name without extension
      audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]
      # new file name with folder route
      file_name = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')
      wavfile.write(file_name, rate, reduced_noise)

In [19]:

# Load progress
last_wav_file_path = load_progress()
# Generate spectrograms
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        print(folder_path)
        print(last_wav_file_path)
        if last_wav_file_path!= None and extract_numbers_from_string(folder_path) < extract_numbers_from_string(last_wav_file_path):
            continue
        for wav_file_path in glob.glob(os.path.join(folder_path, 'preprocessed*.wav')):
             # Extract the audio file name without extension
            audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]
            print(f'Generating spectrogram for: {wav_file_path}')
            # Save progress
            #save_progress(folder_path)
            
            # load data
            rate, data = wavfile.read(wav_file_path)
            
            # Generate spectrogram
            spectrogram_save_path = f"spectrogram_{audio_file_name}.jpg"
            resized_spectrogram_save_path = os.path.join(folder_path, f"resized_spectrogram_{audio_file_name}.jpg")
            generate_spectrogram(data, rate,  os.path.join(folder_path,spectrogram_save_path), resized_spectrogram_save_path)
            
            # Delete original spectrogram
            #os.remove(spectrogram_save_path)

d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\300_P

Generating spectrogram for: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\300_P\preprocessed_300_AUDIO.wav


MemoryError: Unable to allocate 158. MiB for an array with shape (81061, 256) and data type float64

<Figure size 1000x400 with 0 Axes>